In [9]:
import praw
import pandas as pd
import time
from datetime import datetime

# Initialize the Reddit instance
user_agent = "Reddit_Scrapper 1.0 by /u//Specific-Fix-8451"
reddit = praw.Reddit(
 client_id="-TV7jUr2J0W8pmXILv9YgA",
 client_secret="PD6i9dFlEmvbYatT5dAoLqywmiBZrg",
 user_agent=user_agent
)

# Define the current time and the time one year ago
current_time = int(time.time())
one_year_ago = current_time - (8760 * 3600)  # 8760 hours in a year
one_year_ago_datetime = datetime.utcfromtimestamp(one_year_ago)

# Initialize a list to store posts
all_posts = []

# Define the list of subreddits to traverse
subreddits = ['ManchesterUnited',
 'crystalpalace',
 'nffc',
 'coys',
 'WWFC',
 'chelseafc',
 'ArsenalFC',
 'COYH',
 'Everton',
 'BrightonHoveAlbion',
 'NUFC',
 'Hammers',
 'Brentford',
 'MCFC',
 'fulhamfc',
 'LiverpoolFC',
 'Burnley',
 'avfc',
 'SheffieldUnited',
 'AFCBournemouth']

# Iterate over each subreddit
for subreddit_name in subreddits:
    # Initialize a set to store posts for the current subreddit
    subreddit_posts = set()

    # Fetch posts from the current subreddit within the past year
    subreddit = reddit.subreddit(subreddit_name)
    for submission in subreddit.new(limit=None):
        created_time_utc = datetime.utcfromtimestamp(submission.created_utc)
        
        # Ensure the post is within the past year
        if created_time_utc >= one_year_ago_datetime:
            post_details = (
                subreddit_name,
                submission.title,
                str(submission.author),
                created_time_utc.date(),
                submission.score,
                submission.upvote_ratio,
                submission.num_comments
            )
            subreddit_posts.add(post_details)

    # If no posts are found for the current subreddit, manually add a tuple with zero values
    if not subreddit_posts:
        zero_post_details = (
            subreddit_name,
            "",
            "",
            datetime.now().date(),  # Current date for consistency
            0,  # Zero values for score, upvote ratio, and number of comments
            0,
            0
        )
        subreddit_posts.add(zero_post_details)

    # Add the posts for the current subreddit to the list of all posts
    all_posts.extend(subreddit_posts)

# Print the number of unique posts fetched
print("Number of unique posts:", len(all_posts))


Number of unique posts: 17100


In [10]:
import pandas as pd
df = pd.DataFrame(all_posts)

In [11]:
df.columns = ['name','title', 'author', 'date','upvotes_count', 'upvote_ratio','num_comments']
df

,name,title,author,date,upvotes_count,upvote_ratio,num_comments
0,ManchesterUnited,American traveling to see United,BryEngle,2024-03-20,10,0.92,30
1,ManchesterUnited,What do we think about todays starting 11?,Your-mother7646874,2024-02-11,285,0.95,97
2,ManchesterUnited,How will you score ETH performance overall on ...,long-island-,2024-02-20,10,0.78,102
3,ManchesterUnited,Legit Check for 1990-92 Manchester United Away...,saanch3ster,2024-05-16,5,0.86,4
4,ManchesterUnited,Inside Manchester United's axis of power and h...,TheTelegraph,2024-05-09,49,0.80,115
...,...,...,...,...,...,...,...
17095,AFCBournemouth,How many of the current Bournemouth squad can ...,footballquizzer,2023-09-24,6,0.88,1
17096,AFCBournemouth,AFC Bournemouth sign Luis Sinisterra,fatinternetcat,2023-09-01,9,0.77,0
17097,AFCBournemouth,Semenyo will be in the squad tomorrow 🥳,MasterReindeer,2024-05-03,30,0.98,7
17098,AFCBournemouth,Bournemouth vs Tottenham Hotspur,Panarus-biarmicus,2023-08-26,5,0.86,21


In [12]:
avg = df.groupby('name').upvote_ratio.mean()
avg

name
AFCBournemouth        0.906729
ArsenalFC             0.722934
Brentford             0.889332
BrightonHoveAlbion    0.915603
Burnley               0.886296
COYH                  0.947343
Everton               0.918928
Hammers               0.850642
LiverpoolFC           0.937407
MCFC                  0.895382
ManchesterUnited      0.876635
NUFC                  0.870314
SheffieldUnited       0.884470
WWFC                  0.936251
avfc                  0.813899
chelseafc             0.924512
coys                  0.938093
crystalpalace         0.883709
fulhamfc              0.881473
nffc                  0.897725
Name: upvote_ratio, dtype: float64

In [13]:
df.to_csv('reddit_fc_data.csv',  index=False)